<a href="https://colab.research.google.com/github/ftvalentini/misc-notebooks/blob/master/borges_hipalages.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Buscando hipálages en los cuentos de Borges con NLP
#### Autores: Julián Martinez Correa y Francisco Valentini

En Twitter alguién sugirió recopilar todas las hipálages de la literatura de Borges y [@manuelaristaran sugirió usar alguna técnica de Natural Language Processing](https://twitter.com/manuelaristaran/status/1249775645929934851) para esta tarea.  
Una hipálage es una combinación de palabras en la que a un sustantivo se le atribuye una acción (verbo) o una cualidad (adjetivo) que son propios de otro. Por ejemplo, la expresión "noche unámine" en Las Ruinas Circulares.  
Siguiendo la sugerencia, decidimos abordar de forma exploratoria esta tarea con los siguientes pasos:  
- Restringimos el análisis a pares de palabras sustantivo-adjetivo y a [un compilado de ficciones de Borges **en inglés**](https://posthegemony.files.wordpress.com/2013/02/borges_collected-fictions.pdf)
- Extraemos todos los pares sustantivo-adjetivo de los cuentos usando un [modelo de dependecy parsing pre-entrenado](https://stanfordnlp.github.io/stanza/depparse.html) que reconoce las dependencias gramáticales en cada oración
- Medimos las distancias coseno entre los [word embeddings GloVe](https://nlp.stanford.edu/projects/glove/) de cada par de palabras. Un word embedding es un vector de dimensión *n* que codifica el signifcado de una palabra en relación al resto de las palabras de un vocabulario.
- Rankeamos los pares según la distancia coseno y exploramos rankings alternativos en función de la inspección visual de los resultados. 
<!-- normaliza la distancia por la distancia promedio del sustantivo o adejtivo a los *n* adjetivos o sustantivos más cercanos. -->

La hipótesis es que hay una mayor probabilidad de encontrar hipálages en pares sustantivo-adjetivo con distancia alta entre sus embeddings que en pares con distancia baja. Esto se debe a que **en general** las palabras que conforman las hipálages tienen una similitud semántica baja o al menos no tienden a usarse en el mismo contexto. 

En este sentido, este análisis no nos permite estricamente encontrar hipálages (y aún menos evaluar la bondad de los resultados), sino más bien encontrar pares de sustantivo-adjetivos inusuales, llamativos o interesantes en relación al uso común de la lengua inglesa.

### Descargas y librerías

- Instalamos [Stanza](https://github.com/stanfordnlp/stanza/), la librería desarrollada por Stanford que vamos a usar para encontrar las dependencias gramaticales en los textos. La librería ofrece modelos pre-entrenados para múltiples idiomas para aplicar en el contexto de tareas de NLP (en nuestro caso, dependency parsing). 

In [0]:
!pip install stanza

- Descargamos `PyPDF2` para convertir *Ficciones* de formato PDF a un archivo de texto plano.  

In [0]:
!pip install PyPDF2

- Cargamos las librerías que vamos a necesitar para hacer el análisis.

In [0]:
import numpy as np
import pandas as pd
import os

import stanza

import nltk
from nltk.tokenize import sent_tokenize
import gensim.downloader as gensim_api

import PyPDF2 

- Descargamos los modelos de `stanza` para textos en inglés. Los modelos se guardan por defecto en `/root/stanza_resources`. Esta carga nos llevó poco tiempo porque usamos Google Colab,  pero en otro contexto puede llevar algunos minutos -- lo mismo vale para el resto de instalaciones o descargas.

In [0]:
%%time
stanza.download('en')

- Cargamos los vectores [GloVe](https://nlp.stanford.edu/pubs/glove.pdf), word embeddings ajustados por Stanford. En particular, vamos a usar los vectores de mayor dimensión posible (300) y ajustados con la wikipedia en inglés.

In [0]:
def load_embeddings():
    """ Load GloVe Vectors
        Returns: array of size (vocab_size, embeddings_dim)
    """
    vectors = gensim_api.load("glove-wiki-gigaword-300")
    return vectors

In [0]:
%%time
word_vectors = load_embeddings()

[==============================================----] 93.2% 350.5/376.1MB downloaded

- Descargamos los modelos de tokenización `punkt` para separar textos en oraciones. Solo lo vamos a usar para visualizar el texto de forma más amigable.

In [0]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

- Convertimos el pdf con los cuentos en un archivo de texto. Leemos este archivo y lo almacenamos en `texto` como una gran cadena de caracteres.

In [0]:
def pdf_to_txt(pdf_file, txt_file, pdf_first_page=10):
    """ Convert pdf_file to txt_file with strings
    """
    with open(pdf_file,'rb') as pdf, open(txt_file, 'w') as txt:
        read_pdf = PyPDF2.PdfFileReader(pdf_file)
        number_of_pages = read_pdf.getNumPages()
        for page_number in range(pdf_first_page-1, number_of_pages):  
            page = read_pdf.getPage(page_number)
            page_content = page.extractText()
            txt.write(page_content)

def read_text(file):
    """ Reads file as string
    file can be PDF or txt -- if it is PDF it first transforms to txt
    """
    file_root = os.path.splitext(file)[0]   
    file_ext = os.path.splitext(file)[1] 
    if file_ext == '.pdf':
        pdf_to_txt(file, file_root+'.txt')
    with open(file_root+'.txt', "r", encoding='utf-8') as f:
        texto = f.read() 
    return texto

In [0]:
%%time
input_file = 'borges_collected-fictions.pdf'
# input_file = 'the_library_of_babel.txt'
texto = read_text(input_file)

CPU times: user 7.61 s, sys: 19.1 ms, total: 7.63 s
Wall time: 7.64 s


### Limpieza del texto

Limpiamos el texto de la siguiente manera:  
- Reemplazamos los saltos de linea y cualquier otro carácter de tipo whitespace por espacios no repetidos porque no nos interesan los cambios de párrafo ni de página, sino solamente las oraciones  
- Reemplazamos el carácter Š por --

La documentación de `stanza` sugiere juntar varias oraciones en batches separando cada oración con dos saltos de linea `\n\n` para acelerar el procesamiento de muchos documentos. Sin embargo esto dificultaría rastrear al final del análisis las oraciones donde se encuentran las potenciales hipálages -- decidimos entonces conservar el texto como un solo string y que `stanza` se encargue de la tokenización.  

In [0]:
def clean_texto(texto):
    """Cleans raw text
       Returns: clean string
    """
    clean_text = " ".join(texto.split())
    clean_text = clean_text.replace("Š","—")
    return clean_text

texto = clean_texto(texto)

Veamos los primeros 500 caracteres del texto separados en oraciones para facilitar la visualización:

In [0]:
for s in sent_tokenize(texto[:500]):
    print(s)

A Universal History of Iniquity (1935) I inscribe this book to S.D.
— English, innumerable, and an Angel.
Also: I offer her that kernel of myself that I have saved, somehow— the central heart that deals not in words, traffics not with dreams, and is untouched by time, by joy, by adversities.
Preface to the First Edition The exercises in narrative prose that constitute this book were performed from 1933 to 1934.
They are derived, I think, from my re-readings of Stevenson and Chesterton, from the


### Dependencies parsing

Para realizar tareas de NLP con `stanza` es necesario construir un Pipeline con procesadores acordes al idioma de análisis.
El Pipeline se inicializa por defecto con un dependency parser (`depparse`) para determinar las dependencias sintácticas entre las palabras de una oración dada. Para poder correr este modelo es necesario procesar el texto de antemano con los siguientes procesadores:
- `tokenize`: separa el documento en oraciones
- `pos`: identifica el rol de cada palabra en la oración con un modelo de Part-of-Speech (POS) tagging pre-entrenado
- `lemma`: halla el lema correspondiente de cada palabra (la forma que representa las posibles formas flexionadas -- por ejemplo, "walk" es el lema de "walking")

Por defecto estos procesadores se ejecutan antes de `depparse`, el cual usa la información generada en los pasos precedentes para identificar las relaciones sintátictas. Sugerimos revisar [la documentación de `stanza`](https://stanfordnlp.github.io/stanza/pipeline.html) para conocer más detalles. 

In [0]:
%%time
nlp = stanza.Pipeline('en', verbose=False)
docs = nlp(texto)

CPU times: user 40min 39s, sys: 27.3 s, total: 41min 6s
Wall time: 41min 8s


Para identificar las relaciones sustantivo-adjetivo es necesario conservar las relaciones de tipo "amod" (*adjectival modifier*). Para cada relación identificada conservamos el índice de la oración para poder rastrearla más adelante.

In [0]:
def find_dependencies(doc):
    """Find amod dependencies in one parsed doc with one or more sentences 
    Returns: list of dependencies with (sentence_idx, head, dependent)
    """
    deps = list()
    for sent_idx, sent in enumerate(doc.sentences):
        id2word = {word.id: word.text for word in sent.words}
        deps += [(sent_idx, id2word[str(word.head)], word.text) \
                for word in sent.words if word.deprel=='amod']
    return deps
    
deps = find_dependencies(docs)

Veamos los tres primeros y los tres últimos pares hallados: 

In [0]:
print(deps[:3])
print(deps[-3:])

[(2, 'heart', 'central', 0.734546571969986), (4, 'prose', 'narrative', 0.3734986186027527), (5, 'films', 'first', 0.7076624929904938)]
[(10788, 'layers', 'Everlasting'), (10793, 'aesthetics', 'literary', 0.6261562705039978), (10793, 'others', 'many', 0.2628936767578125)]


## Word embeddings

Para cada par de palabras calculamos la distancia entre sus embeddings. Si no existe el word embedding para una palabra, no calculamos la distancia. Finalmente guardamos los resultados en un DataFrame. 

In [0]:
# palabras disponibles en GloVe
vocab = word_vectors.vocab.keys()
print(len(vocab))

In [0]:
%%time
for i, dep in enumerate(deps):
    _words = dep[1:] 
    if set(_words).issubset(vocab):
        deps[i] = dep + (word_vectors.distance(*_words),)

CPU times: user 11min 50s, sys: 2.76 s, total: 11min 53s
Wall time: 11min 53s


In [0]:
deps_df = pd.DataFrame(deps, columns=['sent_idx', 'noun', 'adj', 'dist']) 
deps_df.sort_values(by=['dist'], inplace=True, ascending=False)
deps_df.dropna(inplace=True)
print(deps_df.shape)
print(deps_df.head(20))

(10087, 4)
      sent_idx          noun           adj      dist
4758      3707   schismatics           new  1.307657
9150      8523  trivialities           new  1.283182
898        629     anathemas         first  1.278724
6918      5649      hecatomb        public  1.271595
6231      5087        karats          good  1.262450
4615      3576       paisano          same  1.247534
6212      5072  alexandrines          long  1.247089
3949      3017       impiety          near  1.246878
1731      1333       milonga          same  1.242564
7118      5864           men   punctilious  1.235335
9740      9394        series  misfortunate  1.232602
3810      2864          past    modifiable  1.231535
656        424        battle    indiscreet  1.231510
6337      5186        center     ineffable  1.231363
7123      5866     forewords          long  1.224867
2694      1969      hexagons        native  1.221834
1175       897    plastering        former  1.221735
8908      8091        hatpin       

In [0]:
print(deps_df.tail(10))

       sent_idx      noun   adj      dist
7249       6016  everyone  else  0.187325
7429       6407  everyone  else  0.187325
8204       7228    anyone  else  0.185691
5972       4893    anyone  else  0.185691
6194       5068    anyone  else  0.185691
1080        792  anything  else  0.167966
6669       5463  somebody  else  0.165844
10446     10059    nobody  else  0.156148
7342       6225   anybody  else  0.154146
6667       5456   anybody  else  0.154146


Por inspección visual notamos un patrón común en varios pares de palabras con distancia alta: la combinación de un sustantivo relativamente raro (como *schismatics* o *anathemas*) y un adjetivo relativamente común (como *new* o *first*).  
Una hipótesis es que estos pares tienen una distancia alta no neceseriamente por la distancia semántica entre sustantivo-adjetivo sino más bien por la rareza relativa del sustantivo, que se encuentra relativamente muy alejado del resto de las palabras.         
Decidimos descartar estos casos del análisis eliminando los pares donde se combina un sustantivo poco frecuente con un adjetivo muy frecuente.  
Para esto usamos el ranking de las palabras según frecuencia almacenado en `word_vectors.vocab` en el atributo `count`. Este indicador ordena a las palabras según la frecuencia observada durante el entrenamiento de GloVe, de modo que cuánto más común es la palabra, más alto es.       

In [0]:
print(word_vectors.vocab['house'].count)
print(word_vectors.vocab['schismatics'].count)

399834
130001


In [0]:
# ranking as is
deps_df["noun_freq_rank"] = deps_df["noun"].apply(lambda x: word_vectors.vocab[x].count)
deps_df["adj_freq_rank"] = deps_df["adj"].apply(lambda x: word_vectors.vocab[x].count)
# ranking as percentile
deps_df["noun_freq_rank_pct"] = deps_df["noun_freq_rank"].rank(pct=True)
deps_df["adj_freq_rank_pct"] =  deps_df["adj_freq_rank"].rank(pct=True)

Observamos los 10 sustantivos menos frecuentes:

In [0]:
deps_df[['noun','adj','noun_freq_rank_pct']].sort_values(by='noun_freq_rank_pct' \
    , ascending=True).head(10)

,noun,adj,noun_freq_rank_pct
6213,alexandrines,formless,0.000149
6212,alexandrines,long,0.000149
7802,truco,handed,0.000297
9893,thingamajig,magical,0.000397
5155,effusiveness,exaggerated,0.000496
6231,karats,good,0.000595
6981,inabilities,phonetic,0.000744
6980,inabilities,own,0.000744
6918,hecatomb,public,0.000892
199,cheroot,thoughtful,0.000991


...y los 10 adjetivos más frecuentes:

In [0]:
deps_df[['noun','adj','adj_freq_rank_pct']].sort_values(by='adj_freq_rank_pct' \
    , ascending=False).head(10)

,noun,adj,adj_freq_rank_pct
4758,schismatics,new,0.995787
10407,stories,new,0.995787
7009,education,new,0.995787
10547,government,new,0.995787
10644,government,new,0.995787
4130,man,new,0.995787
10897,economy,new,0.995787
10462,school,new,0.995787
6664,destiny,new,0.995787
2411,system,new,0.995787


Finalmente descartamos un par de palabras dado si el adjetivo está entre el 5% de los más comunes y el sustantivo, entre el 5% de los más raros, tomando como total la cantidad de pares. Estos umbrales son complemetamente arbitrarios pero arrojan buenos resultados por inspección visual.

In [0]:
filtered_df = \
    deps_df.loc[(deps_df['noun_freq_rank_pct'] > 0.05) & (deps_df['adj_freq_rank_pct'] < 0.95)]
print(filtered_df[['sent_idx','noun','adj','dist']].head(20))   

       sent_idx        noun           adj      dist
7118       5864         men   punctilious  1.235335
9740       9394      series  misfortunate  1.232602
3810       2864        past    modifiable  1.231535
656         424      battle    indiscreet  1.231510
6337       5186      center     ineffable  1.231363
2694       1969    hexagons        native  1.221834
1175        897  plastering        former  1.221735
705         466       house   hunchbacked  1.218015
3199       2356      d'être          only  1.215836
2464       1821     opinion  disconsolate  1.202806
767         514     rustler          good  1.199905
2151       1602       books       calmest  1.189988
11394     10759     gauchos     following  1.189452
428         280     request      pitiable  1.189281
6398       5225      friend    lamentable  1.187334
436         283     terrors        former  1.186086
7500       6505        soul   sententious  1.183285
9848       9591        rose     incarnate  1.183260
6101       5

Entre los pares de palabras que son potencialmente hipálages según el criterio de distancia vemos algunas muy interesantes. Podemos rastrearlas entre las oraciones originales usando el `sent_idx`.

In [0]:
# hunchbacked house
print(docs.sentences[466].text)
# calmest books
print(docs.sentences[1602].text)

Sometimes, from the garret window of some hunchbacked house near the water, a woman would dump a bucket of ashes onto the head of a passerby.
This technique fills the calmest books with adventure.


**FIN**